In [1]:
import git
g = git.cmd.Git('/root/Projects/Assignment-Projects')
g.pull()

'Already up-to-date.'

In [1]:
from Sparker.SparkLogProcesser.SparkLogFileHandler import setSparkContext
from pyspark import SparkContext
sc = SparkContext() 
setSparkContext(sc)

In [2]:
from Sparker.Logic.LogicTests import *
day2Logs = readParsedLogsFromHDFS(entireDayParsedLogsFolder2)

NoExistsOnUCSC/Parsed/2016-09-28_parsed has been read by 2017-05-20 04:54:38.763223


In [3]:
from MainSrc.PythonVersionHandler import *
from paths import *

In [4]:
from Sparker.Logic.LogicTests import *
from Sparker.Logic.Trainer import *
#trainPairWiseData(data) 

In [5]:
def printSeparater(c = 3):
    for n in range(c):
        print_('#' * 88)

In [6]:
from sklearn import svm

def evaluateModelOnData(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseData(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    print_('Training with SVMWithSGD...')
    model = SVMWithSGD.train(data, iterations=100)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
    print_('The learned weights:\n' + str(model.weights).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnData(model, data, dataName, modelName)
    return model

def runTrainingExperiment(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseData(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnData(model, testData, 'testData', modelName)

In [7]:
def evaluateModelOnDataSK(model, data, dataName = 'Data', modelName = 'Model'):
    labelsAndPreds = data.map(lambda p: (p.label, model.predict(p.features)))
    truePredictionCount = labelsAndPreds.filter(lambda vp: vp[0] == vp[1]).count()
    instanceCount = data.count()
    accuracy = 100 * truePredictionCount / float(instanceCount)
    print_('\n'+modelName, 'has been evaluated on', dataName, 'by', nowStr())
    print_('The result accuracy is %' + '%.3f\n' % (accuracy))
    return labelsAndPreds

def trainPairWiseDataSK(data, dataName = 'Data', modelName = 'Model', evaluate = True):
    model = svm.SVC()#kernel =  'poly', degree = 3) #[:size][:size]
    size = 5000
    X =  data.map(lambda p: p.features).collect()
    Y =  data.map(lambda p: p.label).collect()
    print_('Training with SVC...')
    model.fit(X, Y)
    print_('\n', modelName, 'has been trained on', dataName, 'by', nowStr())##
   # print_('The learned weights:\n' + str(model.coef_).replace(',', ', ') + '\n')
    if evaluate:
        evaluateModelOnDataSK(model, data, dataName, modelName)
    return model

def runTrainingExperimentSK(trainData, testData, modelName = 'Model', save = True, outputFolder = Day1_iPhone_6_DataFolder):
    model = trainPairWiseDataSK(trainData, 'trainData', modelName)
    if save:
        modelPath = joinPath(outputFolder, modelName)
        try:
            model.save(sc_(), modelPath)
        except Py4JJavaError:
            pass
        print_(modelPath, 'has been saved successfully by', nowStr())
    return evaluateModelOnDataSK(model, testData, 'testData', modelName)

In [8]:
from DeepLearningToRank.DeepDataHandler import readTrainDataFromPickle
#from DeepLearningToRank.DeepTrainer import *

def featureFilter(trainData):
    return trainData.map(lambda x: LabeledPoint(x.label, list(x.features[1:4]) + list(x.features[5:6]) + list(x.features[7:-2])))

def runExperiment(trainData, modelName = 'Model'):
    printSeparater(1)
    testData, trainData = trainData.randomSplit(weights=[0.3, 0.7])
    #testData, trainData = trainData[:int(30*len(trainData)/100)], trainData[int(30*len(trainData)/100):]SKSK
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
    print_('The experiment for', modelName, 'has finished by', nowStr())
    
def trainPairWiseDataTestKeyword2(keyword, location = 'HDFS'):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    if location == 'HDFS':
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_TrainData')
        trainData = readTrainDataFromHDFS(trainDataFile)
    if location == 'oneHot':
        outputFolder = joinPath(joinPath(textTrainDataFolder, 'HDFS'), 'Day1_' + keyword + '_Data')
        trainDataFile = joinPath(outputFolder, inputName + '_' + keyword + '_OneHot_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    else:
        trainDataFile = joinPath(textTrainDataFolder, inputName + '_' + keyword + '_TrainData.txt')
        trainData = readTrainDataFromPickle(trainDataFile)
        trainData = sc_().parallelize(trainData)
        trainData = trainData.map(lambda x: LabeledPoint(x[0], x[1]))
    print_(trainData.first())
    print_('Data from', location)
    preprocessing = scaleTrainData# normalizeTrainData#
    trainData = preprocessing(trainData)
    runExperiment(trainData, modelName = 'ExtendedModel')
    trainData = featureFilter(trainData)
    runExperiment(trainData, modelName = 'FilteredModel')
    printSeparater(2)

In [ ]:
trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')

########################################################################################
########################################################################################
/soe/cicekm/Projects/offlineData/HDFS/Day1_iphone_6_Data/all_day_iphone_6_OneHot_TrainData.txt has been read successfully by 2017-05-20 04:55:14.519168
(1.0,[0.0,15.0,100.0,-4001.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,75.3,-780.0])
Data from oneHot
835692 instances have been scaled by 2017-05-20 04:55:23.390232
########################################################################################
Training with SVMWithSGD...

 ExtendedModel has been trained on trainData by 2017-05-20 04:56:46.508951
The learned weights:
[-0.0157316908411, 0.677082760259, 0.0380149088176, 0.0173701700143, 0.0378760147308, 0.00101568678374, 0.0982604027386, -0.0994856771675, 0.000235646933896, -0.000813688330349, -0.439061732166, 0.439983225479, 0.000678541595546, 0.00113851156813, 

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45905)
Traceback (most recent call last):
  File "/soe/cicekm/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2847, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e7f55c000e75>", line 1, in <module>
    trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
  File "<ipython-input-8-32cbee9c9634>", line 37, in trainPairWiseDataTestKeyword2
    runExperiment(trainData, modelName = 'ExtendedModel')
  File "<ipython-input-8-32cbee9c9634>", line 11, in runExperiment
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
  File "<ipython-input-6-d46f8e893464>", line 22, in runTrainingExperiment
    model = trainPairWiseData(trainData, 'trainData', modelName)
  File "<ipython-input-6-d46f8e893464>", line 18, in trainPairWiseData
    evaluateModelOnData(model, data, dataName, modelName)
  File "<ipyth

ConnectionRefusedError: [Errno 111] Connection refused
Traceback (most recent call last):
  File "/soe/cicekm/anaconda3/lib/python3.5/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/soe/cicekm/anaconda3/lib/python3.5/socketserver.py", line 344, in process_request
    self.finish_request(request, client_address)
  File "/soe/cicekm/anaconda3/lib/python3.5/socketserver.py", line 357, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/soe/cicekm/anaconda3/lib/python3.5/socketserver.py", line 684, in __init__
    self.handle()
  File "/soe/cicekm/bin/spark-1.6.0-bin-hadoop2.6/python/pyspark/accumulators.py", line 235, in handle
    num_updates = read_int(self.rfile)
  File "/soe/cicekm/bin/spark-1.6.0-bin-hadoop2.6/python/pyspark/serializers.py", line 545, in read_int
    raise EOFError
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45905)
T

ConnectionRefusedError: [Errno 111] Connection refused
EOFError


----------------------------------------


...

ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45905)
Traceback (most recent call last):
  File "/soe/cicekm/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2847, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e7f55c000e75>", line 1, in <module>
    trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
  File "<ipython-input-8-32cbee9c9634>", line 37, in trainPairWiseDataTestKeyword2
    runExperiment(trainData, modelName = 'ExtendedModel')
  File "<ipython-input-8-32cbee9c9634>", line 11, in runExperiment
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
  File "<ipython-input-6-d46f8e893464>", line 22, in runTrainingExperiment
    model = trainPairWiseData(trainData, 'trainData', modelName)
  File "<ipython-input-6-d46f8e893464>", line 18, in trainPairWiseData
    evaluateModel

ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45905)
Traceback (most recent call last):
  File "/soe/cicekm/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2847, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e7f55c000e75>", line 1, in <module>
    trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
  File "<ipython-input-8-32cbee9c9634>", line 37, in trainPairWiseDataTestKeyword2
    runExperiment(trainData, modelName = 'ExtendedModel')
  File "<ipython-input-8-32cbee9c9634>", line 11, in runExperiment
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
  File "<ipython-input-6-d46f8e893464>", line 22, in runTrainingExperiment
    model = trainPairWiseData(trainData, 'trainData', modelName)
  File "<ipython-input-6-d46f8e893464>", line 18, in trainPairWiseData
    evaluateModel

ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45905)
Traceback (most recent call last):
  File "/soe/cicekm/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2847, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e7f55c000e75>", line 1, in <module>
    trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
  File "<ipython-input-8-32cbee9c9634>", line 37, in trainPairWiseDataTestKeyword2
    runExperiment(trainData, modelName = 'ExtendedModel')
  File "<ipython-input-8-32cbee9c9634>", line 11, in runExperiment
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
  File "<ipython-input-6-d46f8e893464>", line 22, in runTrainingExperiment
    model = trainPairWiseData(trainData, 'trainData', modelName)
  File "<ipython-input-6-d46f8e893464>", line 18, in trainPairWiseData
    evaluateModel

ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:45905)
Traceback (most recent call last):
  File "/soe/cicekm/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2847, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-e7f55c000e75>", line 1, in <module>
    trainPairWiseDataTestKeyword2('iphone 6', 'oneHot')
  File "<ipython-input-8-32cbee9c9634>", line 37, in trainPairWiseDataTestKeyword2
    runExperiment(trainData, modelName = 'ExtendedModel')
  File "<ipython-input-8-32cbee9c9634>", line 11, in runExperiment
    runTrainingExperiment(trainData, testData, modelName = modelName, save = False)
  File "<ipython-input-6-d46f8e893464>", line 22, in runTrainingExperiment
    model = trainPairWiseData(trainData, 'trainData', modelName)
  File "<ipython-input-6-d46f8e893464>", line 18, in trainPairWiseData
    evaluateModel

ConnectionRefusedError: [Errno 111] Connection refused


In [13]:
import pickle
def savePairWisetrainData(keyword):
    printSeparater(2)
    keyword = keyword.replace(' ', '_')
    inputName = 'all_day'
    outputFolder = joinPath(HDFSDataFolder, 'Day1_' + keyword + '_Data')
    dataTypes = ['_TrainData', '_labeledPairs', '_journey_products']
    for typ in dataTypes:
        fileName = inputName + '_' + keyword + typ
        trainDataFile = joinPath(outputFolder, fileName)
        trainData = sc_().textFile(trainDataFile)
        f = joinPath('/root/Projects/TrainData6', fileName)
        trainData = trainData.collect()
        fp = open(f, "wb")   #Pickling
        pickle.dump(trainData, fp)
        print_(f, 'has been saved successfully by', nowStr())

In [10]:
def trainPairWiseDataTest():
    keywords = ['iphone 6', 'jant', 'nike air max', 'tv unitesi', 'kot ceket', 'camasir makinesi', 'bosch']
    for keyword in keywords:
        #trainPairWiseDataTestKeyword2(keyword)readTrainDataFromHDFS
        savePairWisetrainData(keyword)

In [11]:
def main():
    printSeparater()
    print_('%s:' % nowStr(), 'Running on osldevptst02:Jupyter...')

    trainPairWiseDataTest()

    print_('%s:' % nowStr(), 'DONE')
    printSeparater()


In [15]:
main()

########################################################################################
########################################################################################
########################################################################################
2017-05-09 15:09:16.404133: Running on osldevptst02:Jupyter...
########################################################################################
########################################################################################
/root/Projects/TrainData6/all_day_tv_unitesi_TrainData has been saved successfully by 2017-05-09 15:09:35.835378
/root/Projects/TrainData6/all_day_tv_unitesi_labeledPairs has been saved successfully by 2017-05-09 15:09:56.224687
/root/Projects/TrainData6/all_day_tv_unitesi_journey_products has been saved successfully by 2017-05-09 15:09:58.585784
2017-05-09 15:09:58.589724: DONE
########################################################################################
#####################